Süper bir kurgu yapmışsın. Aşağıda kodundaki her adımı, “neden böyle?” sorularını tek tek açıklıyorum; ardından verdiğin örnek satırlar ile **tahmin alma**, **modeli kaydetme/yükleme** ve **yeni veriyle devam eğitimi (fine-tune)** için tam kod veriyorum.

---

# Adım Adım Açıklamalar

## 1) `y` için `.values` kullandık, `X` için kullanmadık — neden?

* `y = dataset["Injury_Next_Season"].values` ile **hedefi** `numpy` dizisine çeviriyorsun. Sklearn/Keras ikisiyle de uyumlu.
* `X`’i **DataFrame** olarak tutmak önemli; çünkü **`ColumnTransformer` kolon adlarına göre** (isimle) işlem yapıyor. Eğer başta `X.values` yapsaydın, kolon adları kaybolur, seçtiğin sayısal/kategorik sütun listeleri boşa düşer.
* Özet: **y = array**, **X = DataFrame** kalmalı.

## 2) Neden iki aşamalı bölme: `X_train, X_temp, ...` ve sonra `X_valid, X_test, ...`?

* Eğitim/Doğrulama/Test olarak **3’e bölmek** istiyoruz.
* Sklearn’de tek fonksiyonla 3’lü split yok, bu yüzden:

  1. Önce Train vs Temp (Train %70 – Temp %30)
  2. Sonra Temp’i Valid vs Test (%15 – %15) diye ikiye bölüyoruz.
* `stratify=y` sınıf dağılımını her bölmede korur (denge bozulmaz).

## 3) Neden `fit_transform(train)` ama sadece `transform(valid/test)`?

* **Veri sızıntısı (data leakage)** olmaması için **scaler/ohe/imputer** gibi istatistikleri **sadece train** üzerinde öğrenmeliyiz.
* `preprocess.fit_transform(X_train)`: imputer medyanları, OHE kategori listesi, scaler ortalama-std **train’de fit edilir** ve train’e uygulanır.
* `preprocess.transform(X_valid/X_test)`: valid/test üzerinde **fit edilmez**, sadece **aynı dönüşüm** uygulanır.
* `feature_dim = X_train_p.shape[1]`: OHE’den sonra **özellik sayısı artabilir**; bu nedenle gerçek giriş boyutunu **pipeline çıktısından** ölçmek gerekir. ANN giriş katmanını buna göre kuruyoruz.

## 4) `class_weight` neden hesaplandı?

* Sınıflar **tam dengeli** bile olsa, küçük veri setlerinde veya loss yüzünden **minör dengesizlik etkilerini** telafi etmek iyi sonuç verir.
* `compute_class_weight(..., y=y_train)` sadece **train** dağılımına göre **\[sınıf -> ağırlık]** hesaplar; `model.fit(..., class_weight=...)` ile loss tarafında pozitif/negatif hataları eşitler.
* Veri dengeli ise ağırlıklar genelde 1’e yakın çıkar; dengesiz olsa bile **küçük eforla büyük fark** yaratabilir.

## 5) `build_model` ve “Sequential yapmadık mı?” sorusu

* Aslında **`keras.Sequential`** kullandın. Yani “Sequential yapmadık” değil; **tam olarak Sequential yaptın** 🙂
* Alternatif olarak **Functional API** de kullanılabilirdi (çoklu girdi, yan kol, embedding vs. gerektiğinde).
* Buradaki katmanlar: `Dense(64, relu) -> Dropout -> Dense(32, relu) -> Dropout/2 -> Dense(1, sigmoid)`.

  * **ReLU**: derin ağlarda iyi çalışır.
  * **Dropout**: aşırı öğrenmeyi (overfit) azaltır.
  * **Sigmoid**: ikili sınıflandırma çıkışı (0–1 olasılık).

## 6) `EarlyStopping` ve `ReduceLROnPlateau` (plateau)

* **EarlyStopping**: Val setindeki bir metriği izler (sen **`val_auc`** seçmişsin). Bir süre **iyileşme olmazsa** eğitimi durdurur ve **en iyi ağırlıkları geri yükler** (`restore_best_weights=True`). Overfit’i önler, zamanı kurtarır.
* **ReduceLROnPlateau**: Val loss iyileşmeyi bırakınca **öğrenme oranını düşürür** (`factor=0.5`). Büyük LR ile “takılıp” kalmayı önler, daha küçük adımlarla iyileşme şansı verir.

## 7) `history` nedir?

* `model.fit(...)` **her epoch’taki loss/metric** değerlerini bir sözlükte toplar (`history.history`).
* Bunu `pd.DataFrame`’e çevirip grafiklemek, **overfit/underfit** tanısını kolaylaştırır.

## 8) Eğitim grafikleri kodu ne yapıyor?

* `hist = pd.DataFrame(history.history)` ile train/val **loss** ve **AUC** eğrilerini çiziyorsun.
* Loss düşerken val\_loss yükselirse overfit; her ikisi de düşüyorsa iyi; val düzleşirse LR azaltma/erken durdurma devrede.

## 9) Test değerlendirme (0.5 eşik)

* `y_proba = model.predict(X_test_p).ravel()` → **olasılıklar** (0–1).
* `y_pred = (y_proba >= 0.5).astype(int)` → **etiket** (0/1). 0.5 **varsayılan** bir eşiktir; problemi/öncelikleri göre ayarlanabilir.
* Metrikler:

  * **AUC** (ROC AUC): eşikten bağımsız ayrım gücü.
  * **Accuracy/Precision/Recall/F1**: eşik **seçimine bağlı** sonuçlar.
* `classification_report` ve `confusion_matrix` sınıf bazlı performansı gösterir.

## 10) ROC ve PR eğrileri

* **ROC**: FPR–TPR eğrisi, **AUC** ile özetlenir. Sınıf dağılımı dengeliyken daha açıklayıcı.
* **PR**: Precision–Recall eğrisi, **AP (Average Precision)** ile özetlenir. Pozitif sınıf **nadir** olduğunda daha bilgilendiricidir.

## 11) Eşik (threshold) optimizasyonu

* 0.1–0.9 arasında eşikleri tarayıp **F1’i maksimize** eden eşiği arıyorsun.
* F1, precision/recall dengesini sağlar. Maliyet fonksiyonuna göre **başka bir metrik** de seçilebilirdi (örneğin recall öncelikli ise recall\@threshold’u maksimize etmek).

---

# Uygulama: Örnek Verilerle Tahmin, Model Kaydet/Yükle ve Devam Eğitimi

Aşağıdaki bloklar, **senin kurduğun pipeline ile birebir uyumlu**.
Not: Verdiğin “5 satır” örnekte **4 satır** var; o şekilde kullandım.

## A) Örnek veriden tahmin alma

```python
import pandas as pd
import numpy as np

# 1) Örnek veriyi DataFrame'e çevir
sample_csv = """Age,Height_cm,Weight_kg,Position,Training_Hours_Per_Week,Matches_Played_Past_Season,Previous_Injury_Count,Knee_Strength_Score,Hamstring_Flexibility,Reaction_Time_ms,Balance_Test_Score,Sprint_Speed_10m_s,Agility_Score,Sleep_Hours_Per_Night,Stress_Level_Score,Nutrition_Quality_Score,Warmup_Routine_Adherence,Injury_Next_Season,BMI
22,173,64,Midfielder,11.575308026077138,36,1,77.46027901003853,79.11573817783952,284.48785262294393,91.21247628562678,5.874629899095096,77.59970508640522,8.238293030872281,46.61641520851219,81.47220606471353,1,0,21.383941996057334
18,170,67,Midfielder,12.275869450190173,37,2,72.6344422606009,82.54168794778663,250.57924940551655,87.29407824138767,5.796269352614358,94.4189869374685,8.983737193266625,49.36803674229354,81.05667689014356,1,0,23.18339100346021
22,186,75,Forward,12.254895659208355,12,2,77.0644897841478,75.94363053556577,269.11991825792603,83.4406884700333,5.731208756193826,70.17917629629453,7.229192653844745,43.13280804133722,64.87745688032112,0,1,21.678806798473808
20,172,62,Defender,9.00667756614741,11,1,82.81023161351551,73.8783244426281,226.3764118453216,87.59189360708152,6.220212239929888,83.47382412454475,7.681028808856083,51.528529395795616,89.82474393936592,1,0,20.9572742022715
"""
from io import StringIO
sample_df = pd.read_csv(StringIO(sample_csv))

# 2) Hedefi ayır (varsa). Tahmin alırken hedefe ihtiyacımız yok.
X_new = sample_df.drop(columns=["Injury_Next_Season"])

# 3) Eğitimde fit ettiğin PREPROCESSOR'u kullanarak dönüştür
# preprocess ve model zaten bellekteyse direkt kullan:
X_new_p = preprocess.transform(X_new)

# 4) Olasılık ve etiket tahminleri
y_new_proba = model.predict(X_new_p).ravel()
y_new_pred  = (y_new_proba >= 0.5).astype(int)

print("Olasılıklar:", y_new_proba.round(3))
print("Tahminler  :", y_new_pred.tolist())
```

> Not: Pipeline’daki `OneHotEncoder(handle_unknown="ignore")` sayesinde **yeni bir Position** gelse bile hata almazsın (sıfır vektörü verir).

---

## B) Modeli ve ön-işlemeyi kaydetmek / yüklemek

### Kaydetme

```python
# 1) Keras modeli kaydet
model.save("injury_ann.keras")  # veya 'injury_ann.h5'

# 2) Sklearn preprocessörü kaydet
import joblib
joblib.dump(preprocess, "preprocess.joblib")

# (İsteğe bağlı) Eşik ve versiyon notları:
best_threshold_to_save = 0.5  # istersen hesapladığın t_star'ı kaydet
meta = {"threshold": best_threshold_to_save, "version": "v1.0"}
joblib.dump(meta, "meta.joblib")
```

### Yükleme ve kullanma

```python
from tensorflow import keras
import joblib
import pandas as pd

# 1) Yükle
loaded_model = keras.models.load_model("injury_ann.keras")
loaded_preprocess = joblib.load("preprocess.joblib")
meta = joblib.load("meta.joblib")  # threshold vb.

# 2) Yeni veriyi oku ve dönüştür
X_infer = X_new  # örnek olarak yukarıdaki sample'dan
X_infer_p = loaded_preprocess.transform(X_infer)

# 3) Tahmin
proba = loaded_model.predict(X_infer_p).ravel()
pred  = (proba >= meta["threshold"]).astype(int)
print("proba:", proba)
print("pred :", pred)
```

---

## C) Yeni veriyle modeli “beslemek” (devam eğitimi)

Burada iki ayrı katman var:

1. **Ön-işleme** (imputer, scaler, OHE):

* **Seçenek-1 (Stabil üretim)**: **Eski preprocessörü dondur** ve **yeni veriyi de aynı dönüşümle** geçir.

  * Avantaj: Üretimde istikrar, kolay yönetim.
  * Dezavantaj: Yeni **kategoriler** (yeni `Position`) OHE’de **öğrenilmez**, “ignore” edilir (vektörleri sıfır kalır).
* **Seçenek-2 (Güncelleme)**: **Eski veri + yeni veriyi birleştirip** preprocessörü **yeniden fit** et.

  * Avantaj: Yeni kategoriler/features dağılımları tanınır.
  * Dezavantaj: Eski modelle birebir aynı giriş boyutu olmayabilir (OHE boyutu değişebilir); **modeli de yeniden eğitmek** gerekir.

2. **Model**:

* **Devam eğitimi (fine-tune)**: Aynı mimari ile, **yüklediğin modeli** yeni verinin **dönüştürülmüş** (preprocess edilmiş) versiyonu üstünde **küçük LR** ile eğitmeye devam edebilirsin.

### Devam eğitimi – Kod

```python
# Varsayım: "loaded_model" ve "loaded_preprocess" yüklendi.
# Yeni veriniz (CSV) var:
new_df = pd.read_csv("football_injury_new.csv")

# Hedefi ayır
y_new = new_df["Injury_Next_Season"].values
X_new = new_df.drop(columns=["Injury_Next_Season"])

# Seçenek-1: preprocessörü dondurup kullan
X_new_p = loaded_preprocess.transform(X_new)

# Devam eğitimi için düşük öğrenme oranı
from tensorflow.keras import optimizers
loaded_model.compile(
    optimizer=optimizers.Adam(learning_rate=5e-4),  # başlangıca göre daha küçük LR
    loss="binary_crossentropy",
    metrics=["accuracy", keras.metrics.AUC(name="auc"),
             keras.metrics.Precision(name="precision"),
             keras.metrics.Recall(name="recall")]
)

# class_weight istersen yine hesapla (yeni eğitim setine göre)
from sklearn.utils.class_weight import compute_class_weight
classes = np.unique(y_new)
cw = compute_class_weight(class_weight="balanced", classes=classes, y=y_new)
cw_dict = {int(c): w for c, w in zip(classes, cw)}

early2 = keras.callbacks.EarlyStopping(
    monitor="val_auc", mode="max", patience=5, restore_best_weights=True, verbose=1
)

# küçük bir validasyon ayır
from sklearn.model_selection import train_test_split
Xn_tr, Xn_va, yn_tr, yn_va = train_test_split(X_new_p, y_new, test_size=0.2, stratify=y_new, random_state=1)

hist2 = loaded_model.fit(
    Xn_tr, yn_tr,
    validation_data=(Xn_va, yn_va),
    epochs=50,
    batch_size=32,
    class_weight=cw_dict,
    callbacks=[early2],
    verbose=1
)

# Güncellenmiş modeli tekrar kaydet
loaded_model.save("injury_ann_finetuned.keras")
```

> **Önemli notlar**
>
> * **Seçenek-2** (preprocessörü yeniden fit etmek) yapacaksan, **X\_train/X\_valid/X\_test** ile **X\_new**’i birleştir, **yeni bir `preprocess.fit`** yap, **ağ giriş boyutu** (OHE nedeniyle) değişebileceği için **modeli sıfırdan** (veya ilk layer’i adapte ederek) yeniden eğit.
> * Üretim hattında genelde **Seçenek-1** tercih edilir; belirli periyotlarda “versiyon yükseltmesi” ile Seçenek-2 planlı yapılır.

---

## D) (İsteğe bağlı) En iyi eşiği kaydedip yüklemek

Tahmin sonrası **karar eşiğini** (ör. `t_star`) kaydedip üretimde kullan:

```python
# Hesaplamıştın:
# t_star, s_star = best_threshold(y_test, y_proba, metric="f1")

import joblib
joblib.dump({"threshold": float(t_star)}, "decision_threshold.joblib")

# Yükleyip kullanma:
th = joblib.load("decision_threshold.joblib")["threshold"]
pred = (proba >= th).astype(int)
```

---

# Kısa Özet (sorularının çekirdeği)

* **`y.values`**: y’yi array yapmak doğal; **X’i DataFrame** tutmamız **kolon isimleriyle** çalışan `ColumnTransformer` için şart.
* **3’lü split**: Train/Valid/Test ayrımı için iki aşamalı bölme.
* **`fit_transform(train)` / `transform(valid/test)`**: sızıntıyı önlemek ve gerçekçi değerlendirme için.
* **`feature_dim`**: OHE sonrası gerçek giriş boyutunu almak için.
* **`class_weight`**: Dengeyi korumak, küçük dengesizlikleri telafi etmek.
* **Sequential**: Zaten Sequential kullandın; Functional API, çoklu giriş/karmaşık mimaride gerekli olur.
* **EarlyStopping & ReduceLROnPlateau**: Overfit’i ve gereksiz epoch’ları engeller, LR’ı dinamik küçültür.
* **`history`**: Eğitim metriklerinin zaman içi kaydı; grafikle teşhis yaparsın.
* **ROC/PR & threshold**: Eşik-agnostik (AUC/AP) analiz + uygulama eşiğini probleme göre optimize etme.

Takıldığın başka nokta varsa veya **embedding’li pozisyon temsili** versiyonunu da (ANN içinde) görmek istersen, onu da tam kodla veririm.


**“Eşik değeri (threshold)”**:
Modelin çıktısı **olasılıktır** (ör. `P(injury=1)` = 0.73). Bu olasılığı **etikete** (0/1) çevirmek için bir **karar eşiği** seçersin.

* Eğer `p >= eşik` ⇒ **1 (İnjury)**, değilse **0**.
* **Varsayılan** genellikle **0.50**’dir, ama bu **sabit** olmak zorunda değil.

Niye önemli?

* **Precision–Recall** dengesini ve **F1**, **hata maliyetlerini** doğrudan etkiler. Örn. “**kaçırmayalım**” (high recall) istiyorsan **eşiği düşürürsün**; “**yanlış alarm vermeyelim**” (high precision) istiyorsan **eşiği yükseltirsin**.
* Eğitim sırasında **model “eşik” öğrenmez**; yalnızca olasılık üretir. **Eşik**, senin **iş kuralın**dır (post-processing karar kuralı).

Niye kaydediyoruz?

* **Tutarlılık**: Üretimde/raporlarda **aynı karar noktasını** kullanmak için (bugün 0.30, yarın 0.50 olmasın).
* **İzlenebilirlik**: “Bu model v1.2, **eşik=0.30** ile çalışıyor” diyebilmek.
* **Esneklik**: Eşiği **modeli yeniden eğitmeden** değiştirebilirsin (ör. sahada çok alarm geliyorsa 0.30→0.40).

Eşiği nasıl seçerim?

* **F1’i maksimize et** (senin koddaki gibi): hem precision hem recall dengesi.
* **Youden’s J / ROC** (TPR–FPR farkını maksimize et): sınıf maliyetleri benzerse iyi başlangıç.
* **İş kuralı hedefi**: “Recall ≥ 0.95 olsun” ya da “Precision ≥ 0.90 olsun” gibi kısıtlarla PR eğrisinden eşik seç.
* **Maliyet tabanlı**: FP, FN maliyetlerini ver; beklenen maliyeti minimize eden eşiği seç.
* **Kalibrasyon**: Olasılıklar iyi kalibre değilse (Platt/Isotonic), önce kalibre et, sonra eşiği seç.

Kısa örnekler (senin yapına uygun):

```python
# 1) F1’i maksimize eden eşik
from sklearn.metrics import f1_score, precision_recall_curve
import numpy as np

def best_threshold_f1(y_true, y_prob):
    ts = np.linspace(0.01, 0.99, 99)
    best_t, best_s = 0.5, -1
    for t in ts:
        s = f1_score(y_true, (y_prob >= t).astype(int), zero_division=0)
        if s > best_s: best_s, best_t = s, t
    return best_t, best_s

t_star, s_star = best_threshold_f1(y_valid, model.predict(X_valid_p).ravel())
print("F1-opt eşik:", round(t_star, 2), "F1:", round(s_star, 3))
```

```python
# 2) "Precision ≥ 0.90" şartıyla en yüksek recall'u veren eşik
prec, rec, thr = precision_recall_curve(y_valid, model.predict(X_valid_p).ravel())
target_prec = 0.90
candidates = np.where(prec[:-1] >= target_prec)[0]  # son eleman thr ile hizalı değil
i = candidates[np.argmax(rec[candidates])] if len(candidates) else np.argmax(prec[:-1])
t_star = thr[i]
print("Precision≥0.90 için eşik:", round(t_star, 2), "Recall:", round(rec[i], 3))
```

Kaydet/Yükle (eşik bir model parametresi değildir; **ayrı meta** olarak tutulur):

```python
import joblib

# kaydetme
joblib.dump({"threshold": float(t_star), "version": "v1.0"}, "decision_threshold.joblib")

# yükleme ve kullanma
meta = joblib.load("decision_threshold.joblib")
th = meta["threshold"]
y_pred = (model.predict(X_test_p).ravel() >= th).astype(int)
```

Özet:

* **Eşik**, olasılık → sınıf dönüşümü için **karar kuralın**; **modelin parçası değildir**.
* **Seçimi**, **hedef metriklerine** ve **iş maliyetlerine** bağlıdır.
* **Kaydetmek**, üretimde **istikrar** ve **tekrar edilebilirlik** sağlar; gerektiğinde **tek başına ayarlanabilir**.
